# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Jason Lee
- Laurent Lee
- Qingyang Xu
- Tianze Zhang
- Yue Jiao


# Group Members IDs

- A########
- A########
- A########
- A########
- A15626544

# Research Question

How does change in GDP per capita correlate to people’s movie genre preferences in USA? 


## Background and Prior Work

Movies are not merely just entertainment, it could let us experience what we could not do in reality and rich emotions. Reports have shown that films could evoke emotions within us. It is evident that individuals have different movie genre preferences, and the preference could be affected by a lot of factors. For example, GDP per capita is a measure of economic strenth, which is related to people’s personal income. In this project, we aim to investigate the correlation between change in GDP per capita and people’s movie genre preferences. We want unbiased data and visualizations to demonstrate this correlation. 

References (include links):
- 1) https://www.scpr.org/news/2014/12/04/48457/what-watching-movies-can-tell-us-about-how-our-bra/ 
- 2) https://www.thebalance.com/gdp-per-capita-formula-u-s-compared-to-highest-and-lowest-3305848

# Hypothesis


Since GDP per capita is directly related to people’s incomes and lower income could cause people to be depressed, we suspect that when the GDP per capita of a certain time period is lower, people prefer to watch more relaxing movies such as comedies or romance, while when the GDP per capita of a certain time period is higher, people prefer to watch more thrilling movies such as horror or action. 


# Dataset(s)

- Dataset Name: movies_metadata.csv
- Link to the dataset: 
- Number of observations:

- Dataset Name: change_gdp.csv
- Link to the dataset:
- Number of observations:

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.


# Setup

In [0]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup

# Data Cleaning

Describe your data cleaning steps here.

In [5]:
# gdp data
## YOUR CODE HERE
## add change in gdp data
data = pd.read_csv('gdp.csv')
data.set_index('Country Name', inplace = True)
data = data.loc['United States', :]
data = data.dropna()
data = data.drop('Country Code')
data = data.drop('Indicator Code')
data = data.drop('Indicator Name')

#rename the dataframe
gdp = pd.DataFrame(data)
gdp = gdp.reset_index()
gdp.columns = ['date','GDP per capita growth (annual %)']
# change the gdp to numeric 
gdp["date"] = pd.to_numeric(gdp["date"])

FileNotFoundError: ignored

In [0]:
gdp

In [0]:
#read in data from .csv
df_movies = pd.read_csv('movies_metadata.csv')

#drop irrelevant columns
df_movies.drop(columns=['homepage', 'id', 'imdb_id', 'original_language', 'overview', 'spoken_languages', 'status', 
                 'tagline', 'poster_path', 'production_countries', 'video', 'production_companies',
                 'adult', 'belongs_to_collection', 'original_title', 'budget', 
                 'revenue', 'runtime', 'vote_average', 'vote_count'], inplace=True)

#rename and move around columns
df_movies.columns = ['genres', 'popularity', 'date', 'title']
df_movies = df_movies[['title', 'date', 'genres', 'popularity']]
#df_movies

In [0]:
#drop rows that do not have data for title or release date
df_movies.dropna(subset=['date', 'title'], inplace=True)
#df

In [0]:
# Convert all inputs to strings and normalize them
def standardize_date_movie(long):
    string = str(long)
    string = string.strip()
    string = string[0:4]
    date = int(string)
   # if(date < 1960 or date > 2017):
  #      string = np.nan
    return string

In [0]:
# Apply standardization and drop any movies out of the date boundary
df_movies["date"] = df_movies["date"].apply(standardize_date_movie)
df_movies.dropna(inplace=True)

#change it to numeric
df_movies["date"] = pd.to_numeric(df_movies["date"])
# change the popularity to numeric 
df_movies["popularity"] = pd.to_numeric(df_movies["popularity"])

#sort the columns
df_movies.sort_values(by=['date'], ascending=True, inplace = True)

# take all movies from 1971 to 2017 and get rid of other ones
df_movies = df_movies.loc[(df_movies["date"] > 1960) & (df_movies["date"] < 2017)]

#genres = df_movies['genres']
#drop empty genres
df_movies = df_movies[~df_movies['genres'].isin(['[]'])]
#set date as index
#df_movies.set_index('date', inplace=True)

df_movies.head()


In [0]:
#funtion to generate new dataframe based on genre
def genre_df(string):
    df = pd.DataFrame(columns=['title', 'date','genres','popularity'])
    for index, row in df_movies.iterrows():
        if string in row['genres']:
            df.loc[len(df.index)] = df_movies.loc[index]
            
    df.drop('genres', axis=1, inplace=True)
    df = df.reset_index()
    df = df.drop(['title'], axis = 1)
    df = df.groupby(['date']).mean()
    df = df.drop('index', axis = 1)
    return df

In [0]:
#get the genres
drama = genre_df('Drama')
romance = genre_df('Romance')
comedy = genre_df('Comedy')
action = genre_df('Action')
thriller = genre_df('Thriller')
adventure = genre_df('Adventure')

In [0]:
drama

In [0]:
# gdp data
## YOUR CODE HERE
## add change in gdp data
data = pd.read_csv('gdp.csv')
data.set_index('Country Name', inplace = True)
data = data.loc['United States', :]
data = data.dropna()
data = data.drop('Country Code')
data = data.drop('Indicator Code')
data = data.drop('Indicator Name')

#rename the dataframe
change_gdp = pd.DataFrame(data)
change_gdp = change_gdp.reset_index()
change_gdp.columns = ['date','GDP per capita growth (annual %)']
# change the gdp to numeric 
change_gdp["date"] = pd.to_numeric(change_gdp["date"])
change_gdp.head()

In [0]:
drama.reset_index()

In [0]:
def merge(df):
    df = df.reset_index()
    df = df.merge(gdp, how='inner')
    #df = df.drop('index', index = 1)
    return df
drama = merge(drama)
romance = merge(romance)
comedy = merge(comedy)
action = merge(action)
thriller = merge(thriller)
adventure = merge(adventure)


In [0]:
#drama.rename(columns={'index':'gdp'},inplace=True)
#drama
#romance

# convert gdp data to float
drama['GDP per capita growth (annual %)'] = pd.to_numeric(change_gdp['GDP per capita growth (annual %)'])
romance['GDP per capita growth (annual %)'] = pd.to_numeric(change_gdp['GDP per capita growth (annual %)'])
comedy['GDP per capita growth (annual %)'] = pd.to_numeric(change_gdp['GDP per capita growth (annual %)'])
action['GDP per capita growth (annual %)'] = pd.to_numeric(change_gdp['GDP per capita growth (annual %)'])
thriller['GDP per capita growth (annual %)'] = pd.to_numeric(change_gdp['GDP per capita growth (annual %)'])
adventure['GDP per capita growth (annual %)'] = pd.to_numeric(change_gdp['GDP per capita growth (annual %)'])

In [0]:
# grpah all of them, and save to variable
drama_graph = drama.plot.scatter('popularity', 'GDP per capita growth (annual %)')
romance_graph = romance.plot.scatter('popularity', 'GDP per capita growth (annual %)')
comedy_graph = comedy.plot.scatter('popularity', 'GDP per capita growth (annual %)')
action_graph = drama.plot.scatter('popularity', 'GDP per capita growth (annual %)')
thriller_graph = romance.plot.scatter('popularity', 'GDP per capita growth (annual %)')
adventure_graph = comedy.plot.scatter('popularity', 'GDP per capita growth (annual %)')

#set titles
drama_graph.set_title("Drama Graph")
romance_graph.set_title("Romance Graph")
comedy_graph.set_title("Comedy Graph")
action_graph.set_title("Action Graph")
thriller_graph.set_title("Thriller Graph")
adventure_graph.set_title("Adventure Graph")
drama_graph

In [0]:
# corrlational analysis
Drama_corr = drama.corr()
Romance_corr = romance.corr()
Comedy_corr = comedy.corr()
Action_corr = action.corr()
Thriller_corr = thriller.corr()
Adventure_corr = adventure.corr()

# put them in one dataframe
Genre_corr = pd.DataFrame()
Genre_corr.ignore_index = True
Genre_corr = Genre_corr.append(pd.DataFrame([Drama_corr["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr = Genre_corr.append(pd.DataFrame([Romance_corr["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr = Genre_corr.append(pd.DataFrame([Comedy_corr["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr = Genre_corr.append(pd.DataFrame([Action_corr["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr = Genre_corr.append(pd.DataFrame([Thriller_corr["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr = Genre_corr.append(pd.DataFrame([Adventure_corr["popularity"]["GDP per capita growth (annual %)"]]))

Genre_corr.index = [0, 1, 2, 3, 4, 5]
Genre_corr.rename(index={0:"Drama", 1:"Romance", 2:"Comedy", 3:"Action", 4:"Thriller", 5:"Adventure" }, inplace=True)
Genre_corr

As the table above indicates, there is very little corrlation between GDP change and movie perference. Our group suspect that this may cause by the fluctation of GDP data: for example, a good fiscal yaer may 3% growth or 4% growth, but they are both good years. Therefore, out group further classify each years into good years and bad years, where good years have a higher GDP change than average and bad year have a lower GDP than average.

In [0]:
# calculate average GDP
average_gdp = drama["GDP per capita growth (annual %)"].mean()

# create a cloumn for each table with good/bad year classfication
# 1.0 means it is a good year

classify_year = lambda gdp: 1.0 if gdp > average_gdp else 0.0
gdp_good_yaers = []

for item in drama["GDP per capita growth (annual %)"]:
    gdp_good_yaers.append(classify_year(item))
    
drama["good_years"] = gdp_good_yaers
romance["good_years"] = gdp_good_yaers
comedy["good_years"] = gdp_good_yaers
action["good_years"] = gdp_good_yaers
thriller["good_years"] = gdp_good_yaers
adventure["good_years"] = gdp_good_yaers


In [0]:
# graph them
drama_graph_good_or_bad = drama[drama.good_years == 1.0]\
     .plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='red', label='good year');
drama[drama.good_years == 0.0]\
.plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='blue', label='bad year',\
      ax = drama_graph_good_or_bad);

romance_graph_good_or_bad = romance[romance.good_years == 1.0]\
     .plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='red', label='good year');
romance[romance.good_years == 0.0]\
.plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='blue', label='bad year',\
      ax = romance_graph_good_or_bad);

comedy_graph_good_or_bad = comedy[comedy.good_years == 1.0]\
     .plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='red', label='good year');
comedy[comedy.good_years == 0.0]\
.plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='blue', label='bad year',\
      ax = comedy_graph_good_or_bad);

action_graph_good_or_bad = action[action.good_years == 1.0]\
     .plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='red', label='good year');
action[action.good_years == 0.0]\
.plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='blue', label='bad year',\
      ax = action_graph_good_or_bad);

thriller_graph_good_or_bad = thriller[thriller.good_years == 1.0]\
     .plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='red', label='good year');
thriller[thriller.good_years == 0.0]\
.plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='blue', label='bad year',\
      ax = thriller_graph_good_or_bad);

adventure_graph_good_or_bad = adventure[adventure.good_years == 1.0]\
     .plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='red', label='good year');
adventure[adventure.good_years == 0.0]\
.plot(kind='scatter', y='popularity', x='GDP per capita growth (annual %)', color='blue', label='bad year',\
      ax = adventure_graph_good_or_bad);

#set titles
drama_graph_good_or_bad.set_title("Drama Graph (good vs bad years)")
romance_graph_good_or_bad.set_title("Romance Graph (good vs bad years)")
comedy_graph_good_or_bad.set_title("Comedy Graph (good vs bad years)")
action_graph_good_or_bad.set_title("Action Graph (good vs bad years)")
thriller_graph_good_or_bad.set_title("Thriller Graph (good vs bad years)")
adventure_graph_good_or_bad.set_title("Adventure Graph (good vs bad years)")

In [0]:
# calculate corrlation separatly for good years
Drama_corr_good_year = drama[drama.good_years == 1.0].corr()
Romance_corr_good_year = romance[romance.good_years == 1.0].corr()
Comedy_corr_good_year = comedy[comedy.good_years == 1.0].corr()
Action_corr_good_year = action[action.good_years == 1.0].corr()
Thriller_corr_good_year = thriller[thriller.good_years == 1.0].corr()
Adventure_corr_good_year = adventure[adventure.good_years == 1.0].corr()

# put them in one dataframe
Genre_corr_good_years = pd.DataFrame()
Genre_corr_good_years.ignore_index = True
Genre_corr_good_years = Genre_corr_good_years.append(pd.DataFrame([Drama_corr_good_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_good_years = Genre_corr_good_years.append(pd.DataFrame([Romance_corr_good_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_good_years = Genre_corr_good_years.append(pd.DataFrame([Comedy_corr_good_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_good_years = Genre_corr_good_years.append(pd.DataFrame([Action_corr_good_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_good_years = Genre_corr_good_years.append(pd.DataFrame([Thriller_corr_good_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_good_years = Genre_corr_good_years.append(pd.DataFrame([Adventure_corr_good_year["popularity"]["GDP per capita growth (annual %)"]]))

Genre_corr_good_years.index = [0, 1, 2, 3, 4, 5]
Genre_corr_good_years.rename(index={0:"Drama", 1:"Romance", 2:"Comedy", 3:"Action", 4:"Thriller", 5:"Adventure" }, inplace=True)
Genre_corr_good_years

In [0]:
# calculate corrlation separatly for bad years
Drama_corr_bad_year = drama[drama.good_years == 0.0].corr()
Romance_corr_bad_year = romance[romance.good_years == 0.0].corr()
Comedy_corr_bad_year = comedy[comedy.good_years == 0.0].corr()
Action_corr_bad_year = action[action.good_years == 0.0].corr()
Thriller_corr_bad_year = thriller[thriller.good_years == 0.0].corr()
Adventure_corr_bad_year = adventure[adventure.good_years == 0.0].corr()

# put them in one dataframe
Genre_corr_bad_years = pd.DataFrame()
Genre_corr_bad_years.ignore_index = True
Genre_corr_bad_years = Genre_corr_bad_years.append(pd.DataFrame([Drama_corr_bad_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_bad_years = Genre_corr_bad_years.append(pd.DataFrame([Romance_corr_bad_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_bad_years = Genre_corr_bad_years.append(pd.DataFrame([Comedy_corr_bad_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_bad_years = Genre_corr_bad_years.append(pd.DataFrame([Action_corr_bad_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_bad_years = Genre_corr_bad_years.append(pd.DataFrame([Thriller_corr_bad_year["popularity"]["GDP per capita growth (annual %)"]]))
Genre_corr_bad_years = Genre_corr_bad_years.append(pd.DataFrame([Adventure_corr_bad_year["popularity"]["GDP per capita growth (annual %)"]]))

Genre_corr_bad_years.index = [0, 1, 2, 3, 4, 5]
Genre_corr_bad_years.rename(index={0:"Drama", 1:"Romance", 2:"Comedy", 3:"Action", 4:"Thriller", 5:"Adventure" }, inplace=True)
Genre_corr_bad_years

In [0]:
# test how much 

# Data Analysis & Results

In [0]:
#merge relaxing genres and thrilling genres into their own dataframes
df_relaxing_merge = pd.concat([drama, comedy, romance])
df_thrilling_merge = pd.concat([action, adventure, thriller])

In [0]:
#OLS for relaxing movies
f0 = "popularity ~ gdp"
outcome_1, predictors_1 = patsy.dmatrices(f0, df_relaxing_merge, return_type = "matrix")
mod_1 = sm.OLS(outcome_1, predictors_1)
res_1 = mod_1.fit()

In [0]:
#OLS for thrilling movies
f0 = "popularity ~ gdp"
outcome_2, predictors_2 = patsy.dmatrices(f0, df_thrilling_merge, return_type = "matrix")
mod_2 = sm.OLS(outcome_2, predictors_2)
res_2 = mod_2.fit()

Include cells that describe the steps in your data analysis.

In [0]:
## YOUR CODE HERE
gdp.plot(use_index=True)

# Ethics & Privacy

The movie sells data and GDP data may contain some personal information such as the name, age, sex, and income, especially when cross-matching with other data available online. We need to filter that unrelated and personal information before we work with the dataset. We will save all the data in the google drive and only the team members will have access to them. There could be analysis bias since 
A. Data Collection
 A.1 Informed consent: If there are human subjects, have they given informed consent, where subjects affirmatively opt-in and have a clear understanding of the data uses to which they consent?
 A.2 Collection bias: Have we considered sources of bias that could be introduced during data collection and survey design and taken steps to mitigate those?
 A.3 Limit PII exposure: Have we considered ways to minimize exposure of personally identifiable information (PII) for example through anonymization or not collecting information that isn't relevant for analysis?
B. Data Storage
 B.1 Data security: Do we have a plan to protect and secure data (e.g., encryption at rest and in transit, access controls on internal users and third parties, access logs, and up-to-date software)?
 B.2 Right to be forgotten: Do we have a mechanism through which an individual can request their personal information be removed?
 B.3 Data retention plan: Is there a schedule or plan to delete the data after it is no longer needed?
C. Analysis
 C.1 Missing perspectives: Have we sought to address blindspots in the analysis through engagement with relevant stakeholders (e.g., checking assumptions and discussing implications with affected communities and subject matter experts)?
 C.2 Dataset bias: Have we examined the data for possible sources of bias and taken steps to mitigate or address these biases (e.g., stereotype perpetuation, confirmation bias, imbalanced classes, or omitted confounding variables)?
 C.3 Honest representation: Are our visualizations, summary statistics, and reports designed to honestly represent the underlying data?
 C.4 Privacy in analysis: Have we ensured that data with PII are not used or displayed unless necessary for the analysis?
 C.5 Auditability: Is the process of generating the analysis well documented and reproducible if we discover issues in the future?
D. Modeling
 D.1 Proxy discrimination: Have we ensured that the model does not rely on variables or proxies for variables that are unfairly discriminatory?
 D.2 Fairness across groups: Have we tested model results for fairness with respect to different affected groups (e.g., tested for disparate error rates)?
 D.3 Metric selection: Have we considered the effects of optimizing for our defined metrics and considered additional metrics?
 D.4 Explainability: Can we explain in understandable terms a decision the model made in cases where a justification is needed?
 D.5 Communicate bias: Have we communicated the shortcomings, limitations, and biases of the model to relevant stakeholders in ways that can be generally understood?
E. Deployment
 E.1 Redress: Have we discussed with our organization a plan for response if users are harmed by the results (e.g., how does the data science team evaluate these cases and update analysis and models to prevent future harm)?
 E.2 Roll back: Is there a way to turn off or roll back the model in production if necessary?
 E.3 Concept drift: Do we test and monitor for concept drift to ensure the model remains fair over time?
 E.4 Unintended use: Have we taken steps to identify and prevent unintended uses and abuse of the model and do we have a plan to monitor these once the model is deployed?

# Conclusion & Discussion

*Fill in your discussion information here*